In [2]:
import numpy as np
import sys
import os
import yaml
import torch

sys.path.append("/Users/jandavidridder/Desktop/Masterarbeit/src/PYTHON/MYCODE")
cfg_path = "/Users/jandavidridder/Desktop/Masterarbeit/src/PYTHON/MYCODE/models/EtOxModel/EtOxModel.yaml"

from models.EtOxModel.EtOxModel import EtOxModel

with open (cfg_path, "r") as f:
    cfg = yaml.safe_load(f)

meta_model = EtOxModel(model_cfg=cfg, state_keys=cfg["states"].get("keys"), input_keys=cfg["inputs"].get("keys"), N_finite_diff=128)

A = meta_model.get_balance_constraint_matrix(num_stacks=2, include_temp_as_zero=True)
element_species_matrix = meta_model.get_element_species_matrix(num_stacks=2, include_temp_as_zero=True)
x_ref = torch.tensor([[100, 100, 0, 0, 0]], dtype=torch.float32)
display(element_species_matrix)
meta_model.get_element_species_matrix(num_stacks=1)

array([[4., 0., 4., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [2., 0., 2., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 1., 1., 2., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 4., 0., 4., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 2., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2., 1., 1., 2., 0.]])

array([[4., 0., 4., 2., 0., 0.],
       [2., 0., 2., 0., 1., 0.],
       [0., 2., 1., 1., 2., 0.]])

In [ ]:
U = torch.cat([2 * torch.eye(A.shape[1], dtype=A.dtype, device=A.device), A.T], dim=1)
L = torch.cat([A, torch.zeros((A.shape[0], A.shape[0]), dtype=A.dtype, device=A.device)], dim=1)
projection_inverse = torch.linalg.inv(torch.cat([U, L], dim=0))
A = torch.tensor(A, dtype=torch.float32)

def forward(x, x_ref):
    v = torch.cat([2 * x, x_ref @ A.T], dim=-1)
    return v @ projection_inverse.T

display(projection_inverse)
display("cond:", torch.linalg.cond(projection_inverse))

tensor([[ 2.2263e-01,  1.0219e-01, -2.2628e-01,  7.2993e-03,  7.2993e-03,
          7.2727e-01, -1.1363e-01, -1.1363e-01],
        [ 1.0219e-01,  2.5182e-01, -2.1898e-02, -1.6058e-01, -1.6058e-01,
         -1.2794e-01,  4.8989e-01,  4.8989e-01],
        [-2.2628e-01, -2.1898e-02,  2.6277e-01, -7.2993e-02, -7.2993e-02,
          6.6330e-01,  1.3131e-01,  1.3131e-01],
        [ 7.2993e-03, -1.6058e-01, -7.2993e-02,  1.3139e-01,  1.3139e-01,
          8.5862e-02,  8.3901e-01, -1.6099e-01],
        [ 7.2993e-03, -1.6058e-01, -7.2993e-02,  1.3139e-01,  1.3139e-01,
          8.5862e-02, -1.6099e-01,  8.3901e-01],
        [ 7.2727e-01, -1.2794e-01,  6.6330e-01,  8.5862e-02,  8.5862e-02,
         -2.0000e+00, -9.2857e-09, -3.0015e-09],
        [-1.1363e-01,  4.8989e-01,  1.3131e-01,  8.3901e-01, -1.6099e-01,
         -9.2857e-09, -2.0000e+00,  1.2115e-08],
        [-1.1363e-01,  4.8989e-01,  1.3131e-01, -1.6099e-01,  8.3901e-01,
         -3.0015e-09,  1.2115e-08, -2.0000e+00]])

'cond:'

tensor(5.8284)

In [3]:
x_allowed = torch.tensor([[80, 65, 10, 20, 20]], dtype=torch.float32)  # output allowed by stoichiometriy 
x_vorbidden = torch.tensor([[80, 65, 0, 0, 0]], dtype=torch.float32)  # output vorbidden by stoichiometriy 

y_allowed = forward(x_allowed, x_ref=x_ref)
y_vorbidden_corrected = forward(x_vorbidden, x_ref=x_ref)

print(y_allowed)
print(y_vorbidden_corrected)


tensor([[8.0000e+01, 6.5000e+01, 1.0000e+01, 2.0000e+01, 2.0000e+01, 1.5259e-05,
         0.0000e+00, 0.0000e+00]])
tensor([[ 83.9416,  78.2847,  10.5839,  10.9489,  10.9489, -20.1349, -29.7470,
         -29.7470]])


In [21]:
(x_ref - x_allowed.numpy()) @ element_species_matrix.T

/var/folders/0w/sn_84jz11_5gb5j3t0rj_fl40000gn/T/ipykernel_87147/3634991507.py:1: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  (x_ref - x_allowed.numpy()) @ element_species_matrix.T


tensor([[0., 0., 0.]], dtype=torch.float64)